In [47]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import glob

# Define file path if stored in Google Drive
base_path = "/content/drive/My Drive/market_share_data/"

# Read all CSV files
file_paths = glob.glob(base_path + "*.CSV")

dataframes = {}
for file in file_paths:
    df = pd.read_csv(file, encoding='utf-8')
    dataframes[file] = df

# Print first few rows
print(dataframes[file_paths[0]].head())
# Standard column names mapping
standard_columns = {
    "Vendor Name": "Vendor",
    "Super Region": "Super_Region",
    "VendorRevenue - USD": "Vendor_Revenue_USD",
    "ConstantCurrency Revenue - USD": "Constant_Currency_Revenue_USD",
}

def clean_service_names(service):
    if pd.isna(service):
        return "Unknown"
    service = service.lower().strip()
    if "application implementation" in service:
        return "Application Implementation"
    elif "managed services" in service:
        return "Managed Services"
    return service

# Apply transformations
for file, df in dataframes.items():
    df.rename(columns=standard_columns, inplace=True)
    if "Service 2" in df.columns:
        df["Service 2"] = df["Service 2"].apply(clean_service_names)
    dataframes[file] = df

print("Transformation complete!")
for file, df in dataframes.items():
    print(f"Columns in {file}: {df.columns.tolist()}")
# Function to clean column names
def clean_column_names(df):
    df.columns = df.columns.str.strip()  # Remove leading/trailing spaces
    df.columns = df.columns.str.replace(r"[^\w\s]", "", regex=True)  # Remove special characters
    df.columns = df.columns.str.replace(" ", "_")  # Replace spaces with underscores
    df.columns = df.columns.str.replace(r"\.\d+", "", regex=True)  # Remove duplicate suffixes (e.g., Vendor.1 → Vendor)
    df.rename(columns={'Vendor1': 'Vendor'}, inplace=True)  # Force rename Vendor1 to Vendor
    return df

# Apply to all DataFrames
for file, df in dataframes.items():
    dataframes[file] = clean_column_names(df)

# Verify cleaned column names again
for file, df in dataframes.items():
    print(f"✅ Cleaned columns in {file}: {df.columns.tolist()}")
# Drop existing table (ONLY do this if starting fresh)
cursor.execute("DROP TABLE IF EXISTS market_share")

# Create a new table with correct column names
cursor.execute("""
CREATE TABLE market_share (
    Year TEXT,
    Super_Region TEXT,
    Region TEXT,
    Country TEXT,
    Vendor TEXT,
    Service_1 TEXT,
    Service_2 TEXT,
    Service_3 TEXT,
    Vertical TEXT,
    Ticker TEXT,
    HQ_Country TEXT,
    Vendor_Revenue_USD FLOAT,
    Constant_Currency_Revenue_USD FLOAT
);
""")
conn.commit()
for file, df in dataframes.items():
    df.to_sql("market_share", conn, if_exists="append", index=False)

conn.commit()
print("✅ Data inserted successfully!")
df_check = pd.read_sql("SELECT * FROM market_share LIMIT 10", conn)
df_check.head()

import plotly.express as px

# Fetch data from SQLite
df = pd.read_sql("SELECT * FROM market_share", conn)

# Plot Market Share Revenue Trends
fig = px.line(df, x="Year", y="Vendor_Revenue_USD", color="Vendor", title="Market Share Revenue Over Time")
fig.show()
df.to_csv("/content/drive/My Drive/market_share_cleaned.csv", index=False)

Output hidden; open in https://colab.research.google.com to view.